In [ ]:
import evaluate
import numpy as np
import optuna
import os
import torch
import torch.nn as nn

from benchmark import PerformanceBenchmark
from datasets import load_dataset
from pathlib import Path
from onnxruntime import GraphOptimizationLevel, InferenceSession, SessionOptions
from onnxruntime.quantization import quantize_dynamic as onnx_quantize_dynamic, quantize_static as onnx_quantize_static, matmul_nbits_quantizer, quant_utils, QuantType
from psutil import cpu_count
from scipy.special import softmax
from torch.quantization import quantize_dynamic
from training import KDTrainer, KDTrainingArguments
from transformers import AutoConfig, AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.convert_graph_to_onnx import convert_pytorch

%load_ext autoreload
%autoreload 2

## Load model and dataset

In [ ]:
# Model
teacher_ckpt = "transformersbook/bert-base-uncased-finetuned-clinc"
student_ckpt = "distilbert-base-uncased"
pipe = pipeline("text-classification", model=teacher_ckpt)

# Dataset
clinc = load_dataset("clinc_oos", "plus")

# Tokenizers
student_tokenizer = AutoTokenizer.from_pretrained(student_ckpt)

# Metrics
accuracy_score = evaluate.load("accuracy")

# Queries
bench_query = "What is the pin number for my account?"

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE", device)

## Inspect dataset

In [ ]:
sample = clinc["test"][42]
print("Sample", sample)

intents = clinc["test"].features["intent"]
print("Label", intents.int2str(sample["intent"]))

## Run sample from dataset through model

In [ ]:
query = [
    "Hey, I'd like to rent a vehicle from Nov 1st to Nov 15th in Paris and I need a 15 passenger van",
    "Please transfer 100 Dollars from my account."
]
results = pipe(query)
label_ids = [intents.str2int(results[0]["label"]), intents.str2int(results[1]["label"])]
print("Results", results)
print("Label IDs", label_ids)

## Run Benchmarks against baseline model

In [ ]:
bench = PerformanceBenchmark(pipeline=pipe, dataset=clinc["test"], query=bench_query, class_labels=intents)
bench.run_benchmark()

## Prepare Dataset

In [ ]:
def tokenize_text(batch):
    return student_tokenizer(batch["text"], truncation=True)

clinc_enc = clinc.map(tokenize_text, batched=True, remove_columns=["text"])
clinc_enc = clinc_enc.rename_column("intent", "labels")
clinc_enc

## Model Training

In [ ]:
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_score.compute(predictions=predictions, references=labels)

batch_size = 16
finetuned_ckpt = "distilbert-base-uncased-finetuned-clinc"

training_args = KDTrainingArguments(
    output_dir=f"../../checkpoints/{finetuned_ckpt}",
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=500,
    alpha=0.12,
    temperature=7,
    weight_decay=0.01,
    push_to_hub=False,
)

id2label = pipe.model.config.id2label
label2id = pipe.model.config.label2id

num_labels = intents.num_classes
student_config = AutoConfig.from_pretrained(student_ckpt, num_labels=num_labels, id2label=id2label, label2id=label2id)

def student_init():
    return AutoModelForSequenceClassification.from_pretrained(student_ckpt, config=student_config).to(device)

trainer = KDTrainer(
    model_init=student_init,
    teacher_model=pipe.model,
    args=training_args,
    train_dataset=clinc_enc["train"],
    eval_dataset=clinc_enc["validation"],
    compute_metrics=compute_metrics,
    tokenizer=student_tokenizer
)

In [ ]:
trainer.train()

In [ ]:
pipe_student = pipeline("text-classification", model=trainer.model, tokenizer=student_tokenizer)
bench_student = PerformanceBenchmark(pipeline=pipe_student, dataset=clinc["test"], class_labels=intents, query=bench_query, optim_type="DistilBERT")
bench_student.run_benchmark()

## Hyperparameter Tuning

In [ ]:
def hp_space(trial):
    return {
        "num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10),
        "alpha": trial.suggest_float("alpha", 0, 1),
        "temperature": trial.suggest_int("temperature", 2, 20)
    }

best_run = trainer.hyperparameter_search(n_trials=20, direction="maximize", hp_space=hp_space)
print("BEST RUN", best_run)

## Model Quantization

In [ ]:
model = trainer.model.to("cpu")
print("Quantize model ...")
model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
print("Quantization done.")
print("Run benchmarks ...")
pipe_quantized = pipeline("text-classification", model=model_quantized, tokenizer=student_tokenizer, device="cpu")
bench_quantized = PerformanceBenchmark(pipeline=pipe_quantized, dataset=clinc["test"], class_labels=intents, query=bench_query, optim_type="DistilBERT")
bench_quantized.run_benchmark()

## Export to ONNX

In [ ]:
os.environ["OMP_NUM_THREADS"] = f"{cpu_count()}"
os.environ["OMP_WAIT_POLICY"] = "ACTIVE"

model_onnx_path = Path("../../checkpoints/export-onnx/distilbert.onnx")
pipe_export = pipeline("text-classification", model=trainer.model.cpu(), tokenizer=student_tokenizer, device="cpu")

convert_pytorch(
    nlp=pipe_export,
    output=model_onnx_path,
    opset=20,
    use_external_format=False,
)

In [ ]:
def create_model_for_provider(model_path, provider="CPUExecutionProvider"):
    options = SessionOptions()
    options.intra_op_num_threads = 1
    options.graph_optimization_level = GraphOptimizationLevel.ORT_ENABLE_ALL
    session = InferenceSession(str(model_path), options, providers=[provider])
    session.disable_fallback()
    return session

onnx_model = create_model_for_provider(model_onnx_path)

## ONNX Inference Pipeline

In [ ]:
class OnnxPipeline:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def __call__(self, query):
        model_inputs = self.tokenizer(query, return_tensors="pt")
        inputs_onnx = {k: v.cpu().detach().numpy() for k, v in model_inputs.items()}
        logits = self.model.run(None, inputs_onnx)[0][0, :]
        probs = softmax(logits)
        pred_idx = np.argmax(probs).item()
        return [{"label": intents.int2str(pred_idx), "score": probs[pred_idx]}]
    
pipe_onnx = OnnxPipeline(onnx_model, student_tokenizer)
pipe(bench_query)

## ONNX Performance Benchmark

In [ ]:
class OnnxPerformanceBenchmark(PerformanceBenchmark):
    def __init__(self, *args, model_path, **kwargs):
        super().__init__(*args, **kwargs)
        self.model_path = model_path

    def compute_size(self):
        size_mb = np.round(Path(self.model_path).stat().st_size / (1024**2), 3)
        return {"size_mb": size_mb}
    

bench_onnx = OnnxPerformanceBenchmark(pipe_onnx, clinc["test"], model_path=model_onnx_path, class_labels=intents, query=bench_query, optim_type="ORT")
bench_onnx.run_benchmark()

## Dynamic Quantzation of ONNX Model

In [ ]:
model_onnx_path_quantized = model_onnx_path.with_name("quantized.onnx")

onnx_quantize_dynamic(
    model_input=model_onnx_path,             # original ONNX
    model_output=model_onnx_path_quantized,  # quantized output
    weight_type=QuantType.QInt8
)

In [ ]:
onnx_model_quantized = create_model_for_provider(model_onnx_path_quantized)
pipe_onnx_quantized = OnnxPipeline(onnx_model_quantized, student_tokenizer)
bench_onnx_quantized = OnnxPerformanceBenchmark(pipe_onnx_quantized, clinc["test"], model_path=model_onnx_path_quantized, class_labels=intents, query=bench_query, optim_type="ORT-quantized")
bench_onnx_quantized.run_benchmark()

## Static Quantization of ONNX Model

In [ ]:
# Load model and perform shape inference
model = quant_utils.load_model_with_shape_infer(model_onnx_path)
model_onnx_path_q4 = model_onnx_path.with_name("quantized_q4.onnx")

# Define quantization config
config = matmul_nbits_quantizer.DefaultWeightOnlyQuantConfig(
    block_size=128,                # must be 16, 32, 64, 128...
    is_symmetric=False,             # True = QInt4; False = QUInt4
    quant_format=quant_utils.QuantFormat.QOperator,
    op_types_to_quantize=["MatMul"]  # optionally also "Gather"
)

# Create quantizer and quantize model
quantizer = matmul_nbits_quantizer.MatMulNBitsQuantizer(
    model=model,
    algo_config=config
)

quantizer.process()

# Save the quantized model
quantizer.model.save_model_to_file(model_onnx_path_q4, use_external_data_format=False)

In [ ]:
onnx_model_q4 = create_model_for_provider(model_onnx_path_q4)
pipe_onnx_q4 = OnnxPipeline(onnx_model_q4, student_tokenizer)
bench_onnx_q4 = OnnxPerformanceBenchmark(pipe_onnx_q4, clinc["test"], model_path=model_onnx_path_q4, class_labels=intents, query=bench_query, optim_type="ORT-quantized")
bench_onnx_q4.run_benchmark()